In [1]:
import pandas as pd
import numpy as np
import json
import collections
import copy
import random
import math
import matplotlib.pyplot as plt
from nlp.preprocessing import (
    clean_text,
    preprocess,
    tokenize,
    preprocess_document,
    tokenize_document,
    get_stopwords, 
    lemmatization_document,
    get_canonical_words,
    spellcheck_document)
from nlp.pos_tagging import (
    get_tokens_tags
)
from nlp.utils import (
    read_json_file,
    plot_histogram,
    get_completetext,
    plot_wordcloud,
    print_statistics,
    groups_frequency_sort)
from nlp.text_statistics import (
    count_tokens,
    unique_tokens
)
from nlp.grouping import (
    get_groups,
    get_groups_size,
    get_unigram_groups,
    get_two_tokens_groups,
    get_first_token_groups,
    get_bigram_groups,
    get_first_two_groups,
    groups_frequency_sort
)
from utils.read_files import (
    get_items)
from item.item_list import (
    ItemList,
    Item
)
from item.spellcheckeropt import SpellcheckerOpt
from item.utils import get_tokens_set
from textpp_ptbr.preprocessing import TextPreProcessing as tpp
from gensim.parsing.preprocessing import (
    strip_multiple_whitespaces,
    strip_non_alphanum,
    strip_punctuation2,
    strip_short)

In [19]:
licitacao_postgres_file = '../dados/dim_licitacao_postgresql.csv'
licitacoes_post_df = pd.read_csv(licitacao_postgres_file, sep=';')

In [21]:
licitacoes_post_comp = list(licitacoes_post_df['seq_dim_licitacao'])

In [3]:
file_hive = '../dados/itens_hive_F02.csv'
hive_df = pd.read_csv(file_hive, sep=';')

In [4]:
file_postgres = '../dados/itens_postgresql_F02.csv'
post_df = pd.read_csv(file_postgres, sep=';')

In [7]:
licitacoes_hive = list(hive_df['seq_dim_licitacao'])
licitacoes_hive.sort()
licitacoes_post = list(post_df['seq_dim_licitacao'])
licitacoes_post.sort()

In [17]:
len(set(licitacoes_post).intersection(set(licitacoes_hive)))

562

In [22]:
len(set(licitacoes_post_comp).intersection(set(licitacoes_hive)))

1518

In [18]:
len(set(licitacoes_post))

164699

In [2]:
file = '../dados/itens_postgresql_F02.csv'
data = pd.read_csv(file, sep=';')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5886477 entries, 0 to 5886476
Data columns (total 5 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   seq_dim_licitacao   int64 
 1   seq_dim_item        int64 
 2   nom_item            object
 3   dsc_unidade_medida  object
 4   nom_item_busca      object
dtypes: int64(2), object(3)
memory usage: 224.6+ MB


In [4]:
items = data[['seq_dim_licitacao', 'seq_dim_item', 'nom_item', 'dsc_unidade_medida', 'nom_item_busca']].values.tolist()

In [5]:
items[:10]

[[795229,
  16661578,
  'Ornamentacao em festividades e homenagens',
  'srv',
  'ORNAMENTACAO EM FESTIVIDADES E HOMENAGENS'],
 [795229, 16662170, 'Servico de Buffet', 'Un', 'SERVICO DE BUFFET'],
 [795230,
  16662346,
  'Estudo do Projeto ref obras de aplicacao de ACM na fachada da Câmara',
  'srv',
  'ESTUDO DO PROJETO REF OBRAS DE APLICACAO DE ACM NA FACHADA DA CAMARA'],
 [795230,
  16662347,
  'Consultoria técnica e acompanhamento de servicos de instalacao de revestimento em painéis de ACM para a fachada frontal e laterais da edificacao da CÂMARA MUNICIPAL DE ITABIRAMG',
  'srv',
  'CONSULTORIA TECNICA E ACOMPANHAMENTO DE SERVICOS DE INSTALACAO DE REVESTIMENTO EM PAINEIS DE ACM PARA A FACHADA FRONTAL E LATERAIS DA EDIFICACAO DA CAMARA MUNICIPAL DE ITABIRAMG'],
 [795230,
  16662348,
  'Estudo do projeto apresentado levantamento de quantitativos especificacoes técnicas de produtos adequados e servicos necessários para reforma dos sanitários e pintura interna da CMIMG',
  'srv',
  'ESTU

In [6]:
right_word = read_json_file('../dados/palavras/right_words_nilc.json')

In [7]:
items_descriptions = []
stopwords_ = get_stopwords()
relevant_stopwords = {'para', 'com', 'nao', 'mais', 'muito', 'so', 'sem', 'mesmo', 'mesma', 'ha', 'haja', 'hajam', 'houver', 'houvera', 'seja', 'sejam', 'fosse', 'fossem', 'forem', 'sera', 'serao', 'seria', 'seriam', 'tem', 'tinha', 'teve', 'tinham', 'tenha', 'tiver', 'tiverem', 'tera', 'terao', 'teria', 'teriam', 'uma', 'mais', 'entre'}
stopwords_ = stopwords_ - relevant_stopwords
canonical_form, word_class = get_canonical_words()

In [8]:
stopwords_

{'a',
 'ao',
 'aos',
 'aquela',
 'aquelas',
 'aquele',
 'aqueles',
 'aquilo',
 'as',
 'ate',
 'até',
 'como',
 'da',
 'das',
 'de',
 'dela',
 'delas',
 'dele',
 'deles',
 'depois',
 'do',
 'dos',
 'e',
 'ela',
 'elas',
 'ele',
 'eles',
 'em',
 'era',
 'eram',
 'eramos',
 'essa',
 'essas',
 'esse',
 'esses',
 'esta',
 'estamos',
 'estao',
 'estas',
 'estava',
 'estavam',
 'estavamos',
 'este',
 'esteja',
 'estejam',
 'estejamos',
 'estes',
 'esteve',
 'estive',
 'estivemos',
 'estiver',
 'estivera',
 'estiveram',
 'estiveramos',
 'estiverem',
 'estivermos',
 'estivesse',
 'estivessem',
 'estivessemos',
 'estivéramos',
 'estivéssemos',
 'estou',
 'está',
 'estávamos',
 'estão',
 'eu',
 'foi',
 'fomos',
 'for',
 'fora',
 'foram',
 'foramos',
 'formos',
 'fossemos',
 'fui',
 'fôramos',
 'fôssemos',
 'hajamos',
 'hao',
 'havemos',
 'hei',
 'houve',
 'houvemos',
 'houveram',
 'houveramos',
 'houverao',
 'houverei',
 'houverem',
 'houveremos',
 'houveria',
 'houveriam',
 'houveriamos',
 'houv

# Preprocessing items descriptions

In [9]:
for item in items:
    licitacao = item[0]
    description = item[2]
    dsc_unidade = item[3]
    price = 0.0
    if type(dsc_unidade) == str:
        dsc_unidade = tpp.remove_accents(dsc_unidade.lower())
    elif math.isnan(dsc_unidade):
        dsc_unidade = ""
    doc = preprocess_document(description, remove_numbers=False, stopwords=stopwords_)
    doc = tokenize_document(doc)
    doc = lemmatization_document(doc, canonical_form)
    doc = spellcheck_document(doc, right_word)
    doc = lemmatization_document(doc, canonical_form)
    items_descriptions.append((doc, licitacao, price, dsc_unidade, description))

In [10]:
len(items_descriptions)

5886477

In [11]:
itemlist = ItemList()
itemlist.load_items_from_list(items_descriptions)

In [12]:
itemlist.save_items('../dados/items_postgresql_F02')